#Подготовка


In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os, sys

from keras.engine import Input
from keras.engine import Model
from keras.layers import Dropout, Dense, Flatten, Conv1D, MaxPooling1D, Concatenate, LSTM, MaxPooling2D
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
import nltk

from timeit import default_timer as timer

import csv
import re
import random
from random import randint

from random import shuffle

import numpy as np
from nltk.corpus import stopwords

# Download russian stop-words
nltk.download('stopwords')

# Download dataset
!wget --continue -O /content/data.csv https://www.dropbox.com/s/xsrdgu293i8ie86/data.csv?dl=0

In [ ]:
cd gdrive/MyDrive/colab/

In [ ]:
import pandas as pd 
  
# Read IND_data.csv into a DataFrame, assigned to df 
months1 = pd.read_csv("months1.csv", sep=';', encoding='cp1251') 
months2 = pd.read_csv("months2.csv", sep=';', encoding='cp1251') 
df = months1.append(months2)
df = df.reset_index()
# Prints the first 5 rows of a DataFrame as default 
df.head() 

# Prints no. of rows and columns of a DataFrame 
df

In [ ]:
from keras.models import model_from_json

K.set_learning_phase(0)
with open(os.path.join('model.json'), 'r') as fp:
    model = model_from_json(fp.read())

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.load_weights(os.path.join('model.h5'))

def format_out(out, dict):
    stringify = np.vectorize(lambda x: '{}: {:0.2f}'.format(dict[x], out[x]))
    return ', '.join(stringify(np.argsort(-out)))
    return ""

def predict(m, word):
    out = m.predict(np.array([word2input(word, 50)]))
    indMax = np.argmax(out[0])
    return SEX_DICT_REVERSE[indMax]

In [ ]:
letters = "@ABCDEFGHIJKLMNOPQRSTUVWXYZАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789.,!? -"
LETTERS_FOR_RANDOM = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789.,!?    -"
dictSize = len(letters)
maxlen = 50
print("Total letters: " + str(dictSize))

SEX_DICT = {'unknown': 0, 'Мужчины': 1, 'Женщины': 2}
SEX_DICT_REVERSE = {0: 'unknown', 1: 'Мужчины', 2: 'Женщины'}


def sparse(n, size):
    out = [0.0] * size
    if int(n) >= size:
        print("{} {}".format(n, size))
    out[int(n)] = 1.0
    return out

def chartoindex(c):
    c = c.upper()
    if (c not in letters):
        print("Incorrect letter: " + c)
        return 0
    return letters.index(c)


def word2input(word, maxsize):
    word = word.upper()
    word = re.sub('[^0-9A-ZА-Я ]+', '', word)
    input = list(map(lambda c: sparse(chartoindex(c), dictSize), word.upper().replace(" ", "")))
    input += [sparse(dictSize - 1, dictSize)] * (maxsize - len(input))
    return list(input)

def loadFile(file):
    with open(file, encoding='utf-8') as fp:
        data = csv.reader(fp, delimiter=',')
        data = list(data)
        return data

def getPatch(data, count):
    print("Preparing dataSet...")
    max_input = 0
    sexOutput = list()
    input = list()
    stopWords = stopwords.words('russian')
    counter = 0

    shuffle(data)
    for row in data:
        if (counter > count):
            break
        if (row[3].strip() in SEX_DICT ):
            max_input = max(max_input, len(row[0]))

            r = randint(0, 10)
            r2 = randint(0, 1)
            if r < 7:
                if r == 0:
                    text = row[1].strip()
                elif r < 4:
                    if (r2 == 0):
                        text = (row[0] + " " + row[1] + " " + row[2]).strip()
                    else:
                        text = (row[1] + " " + row[0] + " " + row[2]).strip()
                elif r < 6:
                    if (r2 == 0):
                        text = (row[0] + " " + row[1]).strip()
                    else:
                        text = (row[1] + " " + row[0]).strip()
                else:
                    if (r2 == 0):
                        text = (row[1] + " " + row[2]).strip()
                    else:
                        text = (row[2] + " " + row[1]).strip()

                if len(text) < maxlen:
                    input.append(word2input(text, maxlen))
                    sexOutput.append(sparse(SEX_DICT[row[3].strip()], len(SEX_DICT)))
            else:
                if r == 7:
                    text = random.choice(stopWords)
                elif r == 8:
                    text = random.choice(stopWords) + " " + random.choice(stopWords)
                elif r == 9: # Random string
                    N = r = randint(2, 15)
                    text = ''.join(random.choices(LETTERS_FOR_RANDOM, k=N))
                else:
                    text = random.choice(stopWords) + " " + random.choice(stopWords) + " " + random.choice(stopWords)
                text = text.strip().upper()
                if len(text) < maxlen:
                    input.append(word2input(text, maxlen))
                    sexOutput.append(sparse(SEX_DICT["unknown"], len(SEX_DICT)))

            counter += 1
            if (counter % 30000 == 0):
                print(text)

    x = np.array(input)
    x = x.astype('float32')
    slices = [int(0.8 * len(x)), len(x)]
    x_train, x_test, _ = np.split(x, slices)
    action_output_train, action_output_test,            _ = np.split(sexOutput, slices)

    return [x_train, x_test,
            action_output_train, action_output_test]

#@Преобразование данных


Добавление пола

In [ ]:
gender = pd.DataFrame()
gender['Клиент'] = df['Клиент'].unique()
gender['Пол'] = gender.apply(lambda x: 'Гость' if str(x.Клиент)=='Гость' else predict(model, str(x.Клиент)), axis = 1)
gender = gender.set_index('Клиент')
gender = gender.to_dict()['Пол']
df['Пол'] = df.apply(lambda x: gender[x.Клиент], axis = 1)

Удаление ненужных столбцов

In [ ]:
df.drop(columns=['ГородМагазина', 'МагазинЗаказа'])

Преобразование номенклатуры 'Доставка'

In [ ]:
delivery_cheque = df[df.Номенклатура == 'Доставка'].НомерЗаказаНаСайте.to_list()
delivery_cheque = pd.DataFrame(data=delivery_cheque)
delivery_cheque[1] = 1
delivery_cheque
df = df.merge(delivery_cheque, left_on = 'НомерЗаказаНаСайте', right_on = 0, how = 'left')
df = df.drop(columns=[0])
df = df.rename(columns={1:'Доставка'})
df = df.fillna(0)

In [ ]:
df

In [ ]:
region = pd.read_excel("goroda.xlsx", sheet_name='зп+население')
df = df.merge(region[['address', 'dolgota']], left_on=['Регион'], right_on=[region.columns.values[1]], how='left')
df = df.rename(columns={'address':'РегионЗона'})
df = df.drop(columns=['dolgota'],axis=1)

In [ ]:
df['РегионЗона'].value_counts()

Добавление параметра производителя

In [ ]:
df['Компания'] = df.apply(lambda x: x.Номенклатура.replace('.', ',').split(',')[0] , axis = 1)

In [ ]:
df['Маржа'] = df['Маржа'].astype(str)
df['Маржа'] = df['Маржа'].apply(lambda x: x.replace(',', '.'))
df['Маржа'] = df['Маржа'].apply(lambda x: x.replace(' ', ''))
df['Маржа'] = df['Маржа'].astype(float)

In [ ]:
df['СуммаСтроки'] = df['СуммаСтроки'].astype(str)
df['СуммаСтроки'] = df['СуммаСтроки'].apply(lambda x: x.replace(',', '.'))
df['СуммаСтроки'] = df['СуммаСтроки'].apply(lambda x: x.replace(' ', ''))
df['СуммаСтроки'] = df['СуммаСтроки'].astype(float)

In [ ]:
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].astype(str)
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].apply(lambda x: x.replace(',', '.'))
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].apply(lambda x: x.replace(' ', ''))
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].astype(float)

In [ ]:
df['СуммаЗаказаНаСайте'] = df['СуммаЗаказаНаСайте'].astype(str)
df['СуммаЗаказаНаСайте'] = df['СуммаЗаказаНаСайте'].apply(lambda x: x.replace(',', '.'))
df['СуммаЗаказаНаСайте'] = df['СуммаЗаказаНаСайте'].apply(lambda x: x.replace(' ', ''))
df['СуммаЗаказаНаСайте'] = df['СуммаЗаказаНаСайте'].astype(float)

In [ ]:
df['Цена'] = df['Цена'].astype(str)
df['Цена'] = df['Цена'].apply(lambda x: x.replace(',', '.'))
df['Цена'] = df['Цена'].apply(lambda x: x.replace(' ', ''))
df['Цена'] = df['Цена'].astype(float)

In [ ]:
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].astype(str)
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].apply(lambda x: x.replace(',', '.'))
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].apply(lambda x: x.replace(' ', ''))
df['ЦенаЗакупки'] = df['ЦенаЗакупки'].astype(float)

Удаление подозрительных строк

In [ ]:
df = df.drop(df[df.Телефон_new == '0'].index)
df = df.drop(df[df.ПричинаОтмены == '1.4 Ошибка выбора, будет новый заказ'].index)
df = df.drop(df[df.ПричинаОтмены == '1.1 Тест, контрольная закупка'].index)
df = df.drop(df[df.Маржа == 0].index)
df = df.drop(df[df.СуммаДокумента == 0].index)
df = df.drop(df[df.Количество == 0].index)
df = df.drop(df[df.НовыйСтатус  == 'Отменен'].index)
df = df.drop(df[df.НовыйСтатус  == 'Отменяется'].index)
df = df.drop(df[df.НовыйСтатус  == 'Расформирован ПВЗ'].index)
df = df.drop(df[df.НовыйСтатус  == 'Принят в ПВЗ'].index)
df = df.drop(df[df.НовыйСтатус  == 'В резерве'].index)
df = df.drop(df[df.НовыйСтатус  == 'Частичный возврат'].index)
df = df.drop(df[df.НовыйСтатус  == 'Скомплектован Регион'].index)
df = df.drop(df[df.НовыйСтатус  == 'Возврат'].index)
df = df.drop(df[df.НовыйСтатус  == 'Отправлен в ПВЗ'].index)
df = df.drop(df[df.НовыйСтатус  == 'Комплектация Регион'].index)
df = df.drop(df[df.НовыйСтатус  == 'Передан в Регион'].index)
df = df[df['РегионЗона'].notna()]
df = df[df['Маржа'].notna()]
df = df[df['Группа2'].notna()]
df = df[df['Группа3'].notna()]
df = df[df['Группа4'].notna()]
df = df[df['Регион'].notna()]
df = df[df['Цена'].notna()]
df = df[df['СуммаСтроки'].notna()]
df = df[df['ЦенаЗакупки'].notna()]
df = df.drop(df[df.Маржа < -1000].index)
df = df.drop(df[df.Маржа > 10000].index)
df = df.drop(df[df.Цена < 10].index)
df = df.drop(df[df.Количество > 20].index)
df = df.drop_duplicates()

In [ ]:
df.shape

#Исследование данных

In [ ]:
df.drop_duplicates(subset=['Телефон_new'])['Пол'].value_counts()

In [ ]:
df['Телефон_new'].value_counts()

In [ ]:
df.drop_duplicates(subset=['НомерЗаказаНаСайте'])['МетодДоставки'].value_counts()

In [ ]:
df['НовыйСтатус'].value_counts()

In [ ]:
df['МаржаПроцентная'] = (df['СуммаСтроки'] - (df['ЦенаЗакупки'] * df['Количество']))/df['СуммаСтроки']

Исследование данных по компаниям

In [ ]:
companies = df['Компания'].value_counts().to_frame()
companies = companies.drop(companies[companies.Компания < 1].index)
companies['Название'] = companies.index
companies = companies.reset_index(drop=True)
companies.rename(columns={"Компания": "ПроданоТоваров"}, inplace=True)

In [ ]:
companies['МаржаСредняя'] = companies.apply(lambda x: df[df.Компания == x.Название].Маржа.mean(axis=0), axis = 1)

In [ ]:
df = df.drop(df[df['МаржаПроцентная'] == -np.inf].index)

In [ ]:
companies.to_csv('компании.csv', sep=';', encoding='cp1251')

In [ ]:
df['Маржа'].astype(float).sum()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

#Сводники

**По месяцам**

*Размещенные товары*

Выручка, маржа, количество

In [ ]:
df.groupby('МесяцДатыЗаказа').sum()

Выручка

In [ ]:
df['СуммаСтроки'].sum()

Маржа

In [ ]:
df['Маржа'].sum()

Количество

In [ ]:
df['Количество'].sum()

Уникальные заказы, уникальные клиенты

In [ ]:
df.groupby('МесяцДатыЗаказа').nunique()

Уникальные заказы, уникальные клиенты

In [ ]:
df.nunique()

Маржа процентная


In [ ]:
df.groupby('МесяцДатыЗаказа').mean()

Маржа процентная


In [ ]:
df['МаржаПроцентная'].mean()

Средний чек

In [ ]:
df.drop_duplicates(subset=['НомерЗаказаНаСайте']).groupby('МесяцДатыЗаказа').mean()

Средний чек

In [ ]:
df.drop_duplicates(subset=['НомерЗаказаНаСайте'])['СуммаЗаказаНаСайте'].mean()

Среднее количество товаров

In [ ]:
df[df['МесяцДатыЗаказа'] == 7].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['МесяцДатыЗаказа'] == 8].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df.groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

*Выкупленные товары*

In [ ]:
df_bought = df[df.КоличествоПроданоКлиенту != 0]
df_bought['Итог'] = df_bought['Цена'] * df_bought['КоличествоПроданоКлиенту']

In [ ]:
df_bought.shape

Выручка, маржа, количество

In [ ]:
df_bought.groupby('МесяцДатыЗаказа').sum()

Выручка

In [ ]:
df_bought['Итог'].sum()

Маржа

In [ ]:
df_bought['Маржа'].sum()

Количество

In [ ]:
df_bought['КоличествоПроданоКлиенту'].sum()

Маржа процентная месяц 7

In [ ]:
((df_bought[df_bought.МесяцДатыЗаказа == 7]['Цена'] * df_bought[df_bought.МесяцДатыЗаказа == 7]['КоличествоПроданоКлиенту'] - (df_bought[df_bought.МесяцДатыЗаказа == 7]['ЦенаЗакупки'] * df_bought[df_bought.МесяцДатыЗаказа == 7]['КоличествоПроданоКлиенту']))/df_bought[df_bought.МесяцДатыЗаказа == 7]['СуммаСтроки']).mean()

Маржа процентная месяц 8


In [ ]:
((df_bought[df_bought.МесяцДатыЗаказа == 8]['Цена'] * df_bought[df_bought.МесяцДатыЗаказа == 8]['КоличествоПроданоКлиенту'] - (df_bought[df_bought.МесяцДатыЗаказа == 8]['ЦенаЗакупки'] * df_bought[df_bought.МесяцДатыЗаказа == 8]['КоличествоПроданоКлиенту']))/df_bought[df_bought.МесяцДатыЗаказа == 8]['СуммаСтроки']).mean()

Маржа процентная

In [ ]:
((df_bought['Цена'] * df_bought['КоличествоПроданоКлиенту'] - (df_bought['ЦенаЗакупки'] * df_bought['КоличествоПроданоКлиенту']))/df_bought['СуммаСтроки']).mean()

Уникальные заказы, уникальные клиенты

In [ ]:
df_bought.groupby('МесяцДатыЗаказа').nunique()

Уникальные заказы, уникальные клиенты

In [ ]:
df_bought.nunique()

Средний чек

In [ ]:
df_bought[df_bought['МесяцДатыЗаказа'] == 7].groupby('НомерЗаказаНаСайте').sum().mean()

Средний чек

In [ ]:
df_bought[df_bought['МесяцДатыЗаказа'] == 8].groupby('НомерЗаказаНаСайте').sum().mean()

Средний чек

In [ ]:
df_bought.groupby('НомерЗаказаНаСайте').sum().mean()

In [ ]:
df[df['МесяцДатыЗаказа'] == 7].groupby('НомерЗаказаНаСайте').sum()['КоличествоПроданоКлиенту'].mean()

In [ ]:
df[df['МесяцДатыЗаказа'] == 8].groupby('НомерЗаказаНаСайте').sum()['КоличествоПроданоКлиенту'].mean()

In [ ]:
df.groupby('НомерЗаказаНаСайте').sum()['КоличествоПроданоКлиенту'].mean()

In [ ]:
(df.groupby('Телефон_new')['Цена'].sum() / df.groupby('Телефон_new')['Количество'].sum())

**По группам**

Выручка, маржа, количество

In [ ]:
df.groupby('Группа2').sum()

Выручка

In [ ]:
df['СуммаСтроки'].sum()

Маржа

In [ ]:
df['Маржа'].sum()

Количество

In [ ]:
df['Количество'].sum()

Уникальные заказы, уникальные клиенты

In [ ]:
df.groupby('Группа2').nunique()

Уникальные заказы, уникальные клиенты

In [ ]:
df.nunique()

Маржа процентная


In [ ]:
df.groupby('Группа2').mean()['МаржаПроцентная']

Маржа процентная


In [ ]:
df['МаржаПроцентная'].mean()

Средний чек

In [ ]:
df.drop_duplicates(subset=['НомерЗаказаНаСайте']).groupby('Группа2').mean()['СуммаЗаказаНаСайте']

Средний чек

In [ ]:
df.drop_duplicates(subset=['НомерЗаказаНаСайте'])['СуммаЗаказаНаСайте'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ДЕТСКОЕ ПИТАНИЕ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ЖЕНСКИЕ ШТУЧКИ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ИГРУШКИ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'КАНЦТОВАРЫ, КНИГИ, ДИСКИ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'КОСМЕТИКА/ГИГИЕНА'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'КРУПНОГАБАРИТНЫЙ ТОВАР'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ОБУВЬ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ПОДГУЗНИКИ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'СОПУТСТВУЮЩИЕ ТОВАРЫ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ТЕКСТИЛЬ, ТРИКОТАЖ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ТОВАРЫ ДЛЯ ЖИВОТНЫХ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df[df['Группа2'] == 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ'].groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

Среднее количество товаров

In [ ]:
df.groupby('НомерЗаказаНаСайте').sum()['Количество'].mean()

*По выкупленным товарам*

Выручка, маржа, количество

In [ ]:
df_bought.groupby('Группа2').sum()

Выручка

In [ ]:
df_bought['Итог'].sum()

Маржа

In [ ]:
df_bought['Маржа'].sum()

Количество

In [ ]:
df_bought['КоличествоПроданоКлиенту'].sum()

Маржа процентная

In [ ]:
df_bought.groupby('Группа2').mean()['МаржаПроцентная']

Маржа процентная

In [ ]:
((df_bought['Цена'] * df_bought['КоличествоПроданоКлиенту'] - (df_bought['ЦенаЗакупки'] * df_bought['КоличествоПроданоКлиенту']))/df_bought['СуммаСтроки']).mean()

Уникальные заказы, уникальные клиенты

In [ ]:
df_bought.groupby('Группа2').nunique()

Уникальные заказы, уникальные клиенты

In [ ]:
df_bought.nunique()

Средний чек

In [ ]:
df_bought[df_bought['Группа2'] == 'ДЕТСКОЕ ПИТАНИЕ'].groupby('Группа2').sum()

Средний чек

In [ ]:
df_bought[df_bought['Группа2'] == 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ'].groupby('НомерЗаказаНаСайте').sum().mean()

Средний чек

In [ ]:
df_bought.groupby('НомерЗаказаНаСайте').sum().mean()

In [ ]:
df[df['Группа2'] == 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ'].groupby('НомерЗаказаНаСайте').sum()['КоличествоПроданоКлиенту'].mean()

In [ ]:
df[df['МесяцДатыЗаказа'] == 8].groupby('НомерЗаказаНаСайте').sum()['КоличествоПроданоКлиенту'].mean()

In [ ]:
df.groupby('НомерЗаказаНаСайте').sum()['КоличествоПроданоКлиенту'].mean()

In [ ]:
(df.groupby('Телефон_new')['Цена'].sum() / df.groupby('Телефон_new')['Количество'].sum())

In [ ]:
display = pd.DataFrame()
display['Id'] = df.drop_duplicates(subset='Телефон_new')['Телефон_new']
display = display.reset_index(drop=True)
df['МаржаПолная'] = df['Маржа'] * df['Количество']
df_bought['МаржаПолнаяВыкупленная'] = df_bought['Маржа'] * df_bought['КоличествоПроданоКлиенту']

In [ ]:
mean_non_dupe = df.drop_duplicates(subset='НомерЗаказаНаСайте').groupby('Телефон_new').mean()
unique = df.groupby('Телефон_new').nunique()
mean =  df.groupby('Телефон_new').mean()
sum = df.groupby('Телефон_new').sum()
non_dupe = df.drop_duplicates(subset='НомерЗаказаНаСайте')

In [ ]:
non_dupe

In [ ]:
mean_non_dupe_bought = df_bought.groupby(['Телефон_new', 'НомерЗаказаНаСайте']).sum()['Итог'].groupby('Телефон_new').mean()
unique_bought = df_bought.groupby('Телефон_new').nunique()
mean_bought =  df_bought.groupby('Телефон_new').mean()
sum_bought = df_bought.groupby('Телефон_new').sum()
group2 = df_bought.groupby(['Телефон_new', 'Группа2']).sum()
group3 = df_bought.groupby(['Телефон_new', 'Группа3']).sum()
type = df_bought.groupby(['Телефон_new', 'Тип']).sum()
self_carry =  df_bought[(df_bought['МетодДоставки'] == 'Самовывоз') | (df_bought['МетодДоставки'] == 'Pick point')].groupby('Телефон_new').sum()['КоличествоПроданоКлиенту'] / df_bought.groupby('Телефон_new').sum()['КоличествоПроданоКлиенту']

In [ ]:
display = display.merge(unique['НомерЗаказаНаСайте'], left_on='Id', right_on='Телефон_new', how = 'inner')
display = display.rename(columns={'НомерЗаказаНаСайте':'КоличествоЧеков'})

In [ ]:
display = display.merge(mean_non_dupe['СуммаЗаказаНаСайте'], left_on='Id', right_on='Телефон_new', how = 'inner')
display = display.rename(columns={'СуммаЗаказаНаСайте':'СреднийЧек'})

In [ ]:
display = display.merge(mean['Количество'], left_on='Id', right_on='Телефон_new', how = 'inner')
display = display.rename(columns={'Количество':'СреднееЧислоТоваровЧека'})

In [ ]:
display = display.merge(sum['СуммаСтроки'], left_on='Id', right_on='Телефон_new', how = 'inner')
display = display.rename(columns={'СуммаСтроки':'Выручка'})

In [ ]:
display = display.merge(sum['Количество'], left_on='Id', right_on='Телефон_new', how = 'inner')
display = display.rename(columns={'Количество':'КоличествоТоваров'})

In [ ]:
display = display.merge(mean['МаржаПолная'], left_on='Id', right_on='Телефон_new', how = 'inner')
display = display.rename(columns={'МаржаПолная':'СредняяМаржа'})

In [ ]:
display = display.merge(unique_bought['НомерЗаказаНаСайте'], left_on='Id', right_on='Телефон_new', how = 'left')
display = display.rename(columns={'НомерЗаказаНаСайте':'КоличествоЧековВыкупленные'})
display = display.fillna(0)

In [ ]:
display = display.merge(mean_non_dupe_bought, left_on='Id', right_on='Телефон_new', how = 'left')
display = display.rename(columns={'Итог':'СреднийЧекВыкупленные'})
display = display.fillna(0)

In [ ]:
display = display.merge(mean['КоличествоПроданоКлиенту'], left_on='Id', right_on='Телефон_new', how = 'left')
display = display.rename(columns={'КоличествоПроданоКлиенту':'СреднееЧислоТоваровЧекаВыкупленные'})
display = display.fillna(0)

In [ ]:
display = display.merge(sum_bought['Итог'], left_on='Id', right_on='Телефон_new', how = 'left')
display = display.rename(columns={'Итог':'ВыручкаВыкупленная'})
display = display.fillna(0)

In [ ]:
display = display.merge(sum['КоличествоПроданоКлиенту'], left_on='Id', right_on='Телефон_new', how = 'left')
display = display.rename(columns={'КоличествоПроданоКлиенту':'КоличествоТоваровВыкупленные'})
display = display.fillna(0)

In [ ]:
display = display.merge(mean_bought['МаржаПолнаяВыкупленная'], left_on='Id', right_on='Телефон_new', how = 'left')
display = display.rename(columns={'МаржаПолнаяВыкупленная':'СредняяМаржаВыкупленная'})
display = display.fillna(0)

In [ ]:
display = display.merge(non_dupe[['РегионЗона', 'Телефон_new']], left_on='Id', right_on='Телефон_new', how = 'left')
display = display.rename(columns={'РегионЗона':'Область'})
display = display.fillna(0)
display = display.drop(columns='Телефон_new')

In [ ]:
display = display.merge(self_carry.to_frame().fillna(0), left_on='Id', right_index=True, how = 'left')
display = display.rename(columns={'КоличествоПроданоКлиенту':'ДоляСамовывоза'})
display = display.fillna(0)

In [ ]:
display

In [ ]:
display = display[display.КоличествоТоваровВыкупленные != 0]

In [ ]:
display['ДоляВыкупленных'] = display['КоличествоТоваровВыкупленные'] / display['КоличествоТоваров']

In [ ]:
for i in df.Тип.unique():
    display[i] = i
    display = display.merge(type['КоличествоПроданоКлиенту'], left_on=['Id', i], right_on=['Телефон_new', 'Тип'], how='left')
    display = display.drop(columns=[i])
    display = display.rename(columns={'КоличествоПроданоКлиенту':i})
    display = display.fillna(0)
    display[i] = display[i] / display['КоличествоТоваровВыкупленные']

In [ ]:
for i in df.Группа2.unique():
    display[i] = i
    display = display.merge(group2['КоличествоПроданоКлиенту'], left_on=['Id', i], right_on=['Телефон_new', 'Группа2'], how='left')
    display = display.drop(columns=[i])
    display = display.rename(columns={'КоличествоПроданоКлиенту':i})
    display = display.fillna(0)
    display[i] = display[i] / display['КоличествоТоваровВыкупленные']

In [ ]:
for i in df.Группа3.unique():
    display[i] = i
    display = display.merge(group3['КоличествоПроданоКлиенту'], left_on=['Id', i], right_on=['Телефон_new', 'Группа3'], how='left')
    display = display.drop(columns=[i])
    display = display.rename(columns={'КоличествоПроданоКлиенту':i})
    display = display.fillna(0)
    display[i] = display[i] / display['КоличествоТоваровВыкупленные']

In [ ]:
display = display.merge(df_bought[['Телефон_new', 'Гео']].drop_duplicates(subset = 'Телефон_new'), left_on='Id', right_on='Телефон_new', how = 'left')
display = display.drop(columns=['Телефон_new'])

In [ ]:
display = display.merge(pd.get_dummies(display['Область']), left_index=True, right_index=True)
display = display.drop(columns=['Область'])

In [ ]:
display

In [ ]:
display_print = display.astype('string')

In [ ]:
def foo(x):
    return x.replace('.', ',')

display_print = display_print.applymap(foo)

In [ ]:
display_print.to_csv('витрина_новая.csv', encoding='cp1251', sep=';')

In [ ]:
list(display.columns[1:114])

In [ ]:
display = display.drop(columns=['Гео'])

In [ ]:
from sklearn.preprocessing import StandardScaler

display_proc = StandardScaler().fit_transform(display[list(display.columns[1:])])

In [ ]:
display.columns[1:116]

In [ ]:
def get_corr_cols(data, treshold):
 corr_matrix = data.corr().abs()
 upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
 to_drop = [column for column in upper.columns if any(upper[column] > treshold)]
 return to_drop

def trimm_correlated(df_in, threshold):
 df_corr = df_in.corr(method='pearson', min_periods=1)
 df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
 un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
 df_out = df_in[un_corr_idx]
 return df_out, list(set(range(189)) - set(un_corr_idx))

In [ ]:
data_drop_corr, exclude = trimm_correlated(pd.DataFrame(data=display_proc), 0.6)
exclude.sort()

In [ ]:
excluded = [list(display.columns)[i] for i in exclude]
excluded

In [ ]:
excluded = list(['СредняяМаржа', 'КоличествоЧеков', 'СреднийЧек', 'СреднееЧислоТоваровЧека',
 'Выручка',
 'КоличествоТоваров',
 'ИНОЕ',
 'КОСМЕТИКА/ГИГИЕНА',
 'ДЕТСКОЕ ПИТАНИЕ',
 'ВЕРХНЯЯ ДЕТСКАЯ ОДЕЖДА',
 'КАНЦТОВАРЫ',
 'ПОДГУЗНИКИ',
 'ПОЛИГРАФИЧЕСКАЯ ПРОДУКЦИЯ',
 'ИГРУШКИ ДЛЯ ДЕВОЧЕК',
 'КОРЗИНЫ, ЯЩИКИ ДЛЯ ИГРУШЕК',
 'ФОТОАЛЬБОМЫ/ФОТОРАМКИ'])

In [ ]:
excluded

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(40, 30))
sns.heatmap(data_drop_corr.corr(), annot=True)
plt.show()

In [ ]:
from sklearn.decomposition import PCA
reduced = PCA(n_components=2).fit_transform(data_drop_corr)

In [ ]:
def optimalK(data, nrefs=3, maxClusters=6):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal

In [ ]:
from sklearn.cluster import KMeans
gapdf, k = optimalK(data_drop_corr)

In [ ]:
pca = PCA(n_components=2)
pca.fit(data_drop_corr)
X_pca_2 = pca.transform(data_drop_corr)
plt.figure(figsize=(9, 6))
plt.scatter(X_pca_2[:, 0], X_pca_2[:, 1], linewidth=2, marker="o")
plt.xticks(size=16)
plt.yticks(size=16)
plt.xlabel(r'$X_{1}$', size=16)
plt.ylabel(r'$X_{2}$', size=16)
plt.title("PCA with 2 components", loc='left', size=16)
plt.grid(b=1)
plt.xlim(-10, 100)
plt.ylim(-30, 30)
plt.show()

In [ ]:
pca = PCA(n_components=1)
pca.fit(data_drop_corr)
X_pca_1 = pca.transform(data_drop_corr)
plt.figure(figsize=(9, 6))
bins = np.linspace(-1.5, 1.5, 61)
plt.hist(X_pca_1[:, 0], bins=bins)
plt.xticks(size=16)
plt.yticks(size=16)
plt.xlabel(r'$X_{1}$', size=16)
plt.ylabel(r'Number of objects', size=16)
plt.title("PCA with 1 component", loc='left', size=16)
plt.grid(b=1)
plt.show()

In [ ]:
plt.plot(gapdf.clusterCount, gapdf.gap, linewidth=3)
plt.scatter(gapdf[gapdf.clusterCount == k].clusterCount, gapdf[gapdf.clusterCount == k].gap, s=250, c='r')
plt.grid(True)
plt.xlabel('Cluster Count')
plt.ylabel('Gap Value')
plt.title('Gap Values by Cluster Count')
plt.show()

In [ ]:


pca = PCA(n_components=50)
clusterer_3 = KMeans(n_clusters=3, random_state=11, n_jobs=-1)
clusterer_3.fit(pca.fit_transform(data_drop_corr))
labels_3 = clusterer_3.predict(pca.fit_transform(data_drop_corr))
pca = PCA(n_components=50)
clusterer_4 = KMeans(n_clusters=4, random_state=11, n_jobs=-1)
clusterer_4.fit(pca.fit_transform(data_drop_corr))
labels_4 = clusterer_4.predict(pca.fit_transform(data_drop_corr))
pca = PCA(n_components=50)
clusterer_5 = KMeans(n_clusters=5, random_state=11, n_jobs=-1)
clusterer_5.fit(pca.fit_transform(data_drop_corr))
labels_5 = clusterer_5.predict(pca.fit_transform(data_drop_corr))

In [ ]:
def plot_clusters_2D(X, labels):
 pca = PCA(n_components=2)
 pca.fit(X)
 X_pca_2 = pca.transform(X)
 plt.figure(figsize=(9, 6))
 for lab in np.unique(labels):
  plt.scatter(X_pca_2[labels==lab, 0], X_pca_2[labels==lab, 1], linewidth=2, marker="x", label=str(lab))
 plt.xticks(size=16)
 plt.yticks(size=16)
 plt.xlabel(r'$X_{1}$', size=16)
 plt.ylabel(r'$X_{2}$', size=16)
 plt.title("PCA with 2 components", loc='left', size=16)
 plt.legend(loc='best', fontsize=16)
 plt.grid(b=1)
 plt.show()

In [ ]:
plot_clusters_2D(data_drop_corr, labels_3)

In [ ]:
plot_clusters_2D(data_drop_corr, labels_4)

In [ ]:
plot_clusters_2D(data_drop_corr, labels_5)

In [ ]:
pca = PCA(n_components=2)
pca.fit(data_drop_corr)
X_pca_2 = pca.transform(data_drop_corr)
plt.figure(figsize=(9, 6))
plt.scatter(X_pca_2[:, 0], X_pca_2[:, 1], linewidth=2, marker="o")
plt.xticks(size=16)
plt.yticks(size=16)
plt.xlabel(r'$X_{1}$', size=16)
plt.ylabel(r'$X_{2}$', size=16)
plt.title("PCA with 2 components", loc='left', size=16)
plt.grid(b=1)
plt.xlim(-10, 15)
plt.ylim(-30, 30)
plt.show()

In [ ]:
display['Кластер'] = labels_5 + 1

In [ ]:
display

In [ ]:
display.drop(columns=excluded, axis = 1).to_csv('витрина.csv', sep=';', encoding='cp1251')